Make pkl file for mmdet3d framework

Training Set in RadarOCC:
{4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 27, 56}

Test Set
{3, 15, 22, 23, 55}

Val SeT
{1,2}

Adveser weather Set (No reliable GT)
{21,46,54}

Modify occ_path, sparse_radar_path with your GT/Sparse radar file path. 

In [ ]:
import glob
kradar_dict = {}
kradar_dict['metadata'] = {'version': 'kradar'}
kradar_dict['infos'] = []
for seq in [4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 27, 56]:

    with open('/mnt/Kradar/K-Radar/{}/info_calib/calib_radar_lidar.txt'.format(seq)) as f:
                lines = f.readlines()
                f.close()
    list_calib = list(map(lambda x: float(x), lines[1].split(',')))
    frame_difference = int(list_calib[0])
    len_occgt = len(glob.glob("/mnt/data/DataSet/K-RadarOOC/train/{}/semantic_occupancy_gt_fov/occupancy_gt_with_semantic_fov*.npy".format(seq)))
    len_radar = len(glob.glob("/mnt/Kradar/K-Radar/{}/radar_zyx_cube/cube_*.mat".format(seq)))-frame_difference
    seq_size = min(len_occgt,len_radar)

    image_files = sorted(glob.glob('/mnt/Kradar/K-Radar/'+str(seq)+'/cam-front/'+ '*.png'))[::3]
    seq_size = min(seq_size,len(image_files))
    print(seq_size)
    for i in range(0,seq_size):
        seq_dict = {}
        seq_dict['occ_path'] = "/mnt/data/DataSet/K-RadarOOC/train/{}/semantic_occupancy_gt_fov/occupancy_gt_with_semantic_fov{}.npy".format(seq,i)
        seq_dict['lidar_path'] ="/mnt/data/DataSet/K-RadarOOC/train/"+str(seq)+"/os2-32/os2-32_{0:05d}.bin".format(i+1)
        seq_dict['radar_path'] = "/mnt/Kradar/K-Radar/"+str(seq)+"/radar_zyx_cube/cube_{0:05d}.mat".format(i+1+frame_difference)
        seq_dict['radar_tensor_path'] = "/mnt/Kradar/K-Radar/"+str(seq)+"/radar_tesseract/tesseract_{0:05d}.mat".format(i+1+frame_difference)
        seq_dict['radar_polar_path'] = "/mnt/18T-Data/kradar/"+str(seq)+"/radar_polar_cube/DRAE_{0:05d}.npy".format(i+1+frame_difference)
        seq_dict['sparse_radar_path'] = "/mnt/18T-Data/kradar/"+str(seq)+"/radar_tensor_8doppler/EAsparse_{0:05d}.npz".format(i+1+frame_difference)
        seq_dict['scene_token'] = seq
        seq_dict['lidar_token'] = i
        seq_dict['cams'] = {'CAM_FRONT': {'data_path': image_files[i],
'cam_intrinsic': np.array([[567.72077648,   0.        , 628.72078   ],
       [  0.        , 577.21369171, 369.30687   ],
       [  0.        ,   0.        ,   1.        ]])
,

                                'lidar2cam': np.array([[ 1.64325268e-04, -9.99390895e-01,  3.48971584e-02, 1.80000000e-01],
       [ 6.72339597e-03,  3.48974743e-02,  9.99368282e-01, -1.30000000e-01],
       [-9.99977384e-01,  7.04059534e-05,  6.72503525e-03, -3.12250226e-17],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]),
                                'sensor2lidar_translation': np.array([-1.80000000e-01,  1.30000000e-01,  3.12250226e-17]),
                                'sensor2lidar_rotation' : np.array([[ 1.64325268e-04,  6.72339597e-03, -9.99977384e-01],
       [-9.99390895e-01,  3.48974743e-02,  7.04059534e-05],
       [ 3.48971584e-02,  9.99368282e-01,  6.72503525e-03]])
                                        }
                            }
        seq_dict['lidar2ego_translation'] = [0,0,0]
        seq_dict['lidar2ego_rotation'] = [1,0,0,0]
        seq_dict['ego2global_translation'] = [0,0,0]
        seq_dict['ego2global_rotation'] = [1,0,0,0]
        seq_dict['timestamp'] = i * 1e16
        seq_dict['prev'] = i - 1
        seq_dict['next'] = i + 1
        seq_dict['lidarseg'] = None
        seq_dict['token'] = seq_dict['lidar_path']
        kradar_dict['infos'].append(seq_dict)

In [24]:
import pickle
with open('kradar_dict_train_doppler8.pkl', 'wb') as f:
    pickle.dump(kradar_dict, f)